# Transfer Learning using VGG16, classified new animals

In [1]:
#Import packages
from keras.applications import VGG16

from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

image_size = 150

#Load the VGG model
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

C:\Installations\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Set directories
base_dir = 'C:\\Users\\vindla\\Downloads\\Cg_DS4\\computer_vision\\animal'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [4]:
# Freeze the layers except the last 2 layers
for layer in conv_base.layers[:-2]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in conv_base.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x00000215F790A748> False
<keras.layers.convolutional.Conv2D object at 0x00000215897E5A90> False
<keras.layers.convolutional.Conv2D object at 0x0000021589918588> False
<keras.layers.pooling.MaxPooling2D object at 0x0000021589EBE7F0> False
<keras.layers.convolutional.Conv2D object at 0x0000021589F11B00> False
<keras.layers.convolutional.Conv2D object at 0x0000021589F22DA0> False
<keras.layers.pooling.MaxPooling2D object at 0x0000021589F31438> False
<keras.layers.convolutional.Conv2D object at 0x0000021589F57710> False
<keras.layers.convolutional.Conv2D object at 0x0000021589F57048> False
<keras.layers.convolutional.Conv2D object at 0x0000021589F795C0> False
<keras.layers.pooling.MaxPooling2D object at 0x0000021589F9DCC0> False
<keras.layers.convolutional.Conv2D object at 0x0000021589FC2208> False
<keras.layers.convolutional.Conv2D object at 0x0000021589FC2D30> False
<keras.layers.convolutional.Conv2D object at 0x0000021589FE4630> False
<keras.

In [5]:
# Setup the data generators
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 20
val_batchsize = 10
test_batchsize = 10
image_size = 150
 
train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(image_size, image_size),
        batch_size=test_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 142 images belonging to 3 classes.
Found 33 images belonging to 3 classes.
Found 43 images belonging to 3 classes.


In [6]:
# Create Model
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(conv_base)
 
# Add new layers
#model.add(layers.Flatten())

model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))  

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 4, 4, 256)         131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 12291     
Total params: 14,858,307
Trainable params: 2,503,427
Non-trainable params: 12,354,880
_________________________________________________________________


In [7]:
# Train the model

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# Fit the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size, 
      verbose=1)
 
# Save the model
model.save('5_3_VGG16.h5')

Epoch 1/30
8/7 [=================================] - 32s 4s/step - loss: 1.1537 - acc: 0.4183 - val_loss: 0.5284 - val_acc: 0.9091
Epoch 2/30
8/7 [=================================] - 32s 4s/step - loss: 0.6525 - acc: 0.7155 - val_loss: 0.3370 - val_acc: 1.0000
Epoch 3/30
8/7 [=================================] - 32s 4s/step - loss: 0.4285 - acc: 0.8099 - val_loss: 0.2417 - val_acc: 1.0000
Epoch 4/30
8/7 [=================================] - 32s 4s/step - loss: 0.4628 - acc: 0.8352 - val_loss: 0.2096 - val_acc: 1.0000
Epoch 5/30
8/7 [=================================] - 32s 4s/step - loss: 0.3079 - acc: 0.8923 - val_loss: 0.1086 - val_acc: 1.0000
Epoch 6/30
8/7 [=================================] - 31s 4s/step - loss: 0.2717 - acc: 0.8923 - val_loss: 0.0824 - val_acc: 1.0000
Epoch 7/30
8/7 [=================================] - 31s 4s/step - loss: 0.2232 - acc: 0.9176 - val_loss: 0.0597 - val_acc: 1.0000
Epoch 8/30
8/7 [=================================] - 31s 4s/step - loss: 0.2299 - a

In [8]:
# Get the predictions from the model using the generator
predictions = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)

5/4 [==================================] - 7s 1s/step


In [31]:
predicted_classes[1:5]

array([0, 0, 0, 0], dtype=int64)